# 入力 1
p. 43

In [1]:
# Word2Vecモデルの初期化
from gensim.models import Word2Vec
model = Word2Vec.load('word2vec.gensim.model')

# 特徴量ベクトルの取得
v1 = model.wv['女王']
v2 = model.wv['王']

# 類似単語の取得
v3 = v1 - v2
for word, score in model.wv.most_similar([v3], topn=10):
    print(word, score)


ナイトクラブ 0.583662748336792
ヌード 0.5530823469161987
スクラッグス 0.5426580905914307
サロン 0.541516125202179
パーティー 0.5277104377746582
ダンサー 0.5097211003303528
ジョー・ミーク 0.5076866745948792
ストリップ 0.5030403733253479
踊り子 0.49707716703414917
ショー 0.49362289905548096


# 入力 2
pp.49-50

In [2]:
import twitter, oauthlib
# ハードコーディングしてもよいがここで指定しても良い
twitter.oauth_client = oauthlib.oauth1.Client(
    '*********************',
    client_secret='******************************************',
    resource_owner_key = '**************************************************',
    resource_owner_secret = '*****************************************'  
)

In [3]:
import time, pandas as pd, mecabwrapper

# 時間計測
start = time.time()

# 取得Tweet数
N = 50
print('Tweet数: {}'.format(N))

# DataFrameを結合する
df = pd.concat([

    #Tweetの本文(text)中の単語を分解したDataFrameを得る。
    mecabwrapper.parseText(tw['text'])
      #日本語TweetをN個取得する。
      for tw in twitter.get_sample_tweet(N, lambda x:x.get('lang') == 'ja')
        
])

# 総単語数はdfの行数
print('総単語数: {}'.format(df.size))

# ユニーク単語数を数える
print('Unique 単語数: {}'.format(df['原型'].nunique()))

# 上位10個のカウントを表示
print(df['原型'].value_counts().head(10))

# 時間計測
end = time.time()
print('経過時間: {} s'.format(end - start))

Tweet数: 50
総単語数: 7248
Unique 単語数: 561
た     25
に     24
@     24
だ     24
て     23
する    18
。     18
が     17
ない    17
の     17
Name: 原型, dtype: int64
経過時間: 10.254461526870728 s


# 入力 3
p. 51

In [4]:
import tweetanalysis
print(tweetanalysis.filter_words(df)['原型'].value_counts().head(10))

する         18
HTTPS      14
co         14
ReTweet    12
なる          7
思う          7
好き          6
できる         4
どう          4
いい          3
Name: 原型, dtype: int64


# 入力 4
p. 54

In [5]:
# 本文では1000としたが、あまりに時間がかかるので50くらいで

N = 50
sample_tweets = twitter.get_sample_tweet(N, tweetanalysis.filter_tweet)
sample_tweets = [*tweetanalysis.parse_tweet_list(sample_tweets)]

In [6]:
N = len(sample_tweets)
word_expectation = tweetanalysis.filter_words(
    pd.concat(sample_tweets)
)['原型'].value_counts(
).to_frame('expectation').loc[
    lambda x:x.index.map(model.__contains__)
]

display(word_expectation.head(10))

,expectation
する,9
俺,5
なる,4
ない,4
垢,3
自分,2
みずき,2
こんな,2
愛す,2
爆撃,2


実際に遊ぶにあたって、集計済みデータを用いたほうが精度が高いと思われる。

In [7]:
import sqlite3

conn = sqlite3.connect('word_expectation.db')
try:
    expectation = pd.read_sql('SELECT * FROM expectation', conn).set_index('word')
finally:
    conn.close()

# 入力 5
p. 57

In [8]:
#解析対象ユーザを指定
screen_name = '******'

tweets_score, N = tweetanalysis.get_tweets_score(
    screen_name,
    expectation,
    model
)
tweets_score.sort_values(
    's
    ascending=False,
    inplace=True
)

print('取得 Tweet数: {}'.format(N))
display(tweets_score.head(10))

取得 Tweet数: 680


,N,expectation,score
する,535,202.803660,276.027095
局所,20,0.005847,221.256419
東工大,20,0.006977,215.907024
改ざん,15,0.005590,163.985173
円グラフ,10,0.000184,161.759854
正しい,30,0.363158,158.471954
最適,20,0.072241,145.180905
比例,15,0.031231,124.306418
値,15,0.033007,123.032107
入試,15,0.034207,122.209923


# 入力 6
p. 59

In [9]:
feature_vector = {
    sn: tweetanalysis.get_feature_vector(
        tweetanalysis.get_tweets_score(sn, expectation, model)[0],
        model
    )
    for sn in ['******', 'ha_chu', 'ariyoshihiroiki']
}

# 単語,コサイン類似度のtupleのlistが返されるので単語だけのlistにしている。

for sn in feature_vector:
    print(sn)
    print([
     w for w, s in model.wv.most_similar([feature_vector[sn]])
    ])

gjhdvhd
['振舞', '受け手', '安全率', '荒らし行為', '物事', '理屈', '矛盾許容論理', 'ディザ', '人身攻撃', '受身']
ha_chu
['大切な人', '好きな人', '飲み会', '恋愛相談', 'お手伝い', 'お客さん', 'お話', '若い人', '人生相談', '追っかけ']
ariyoshihiroiki
['山田花子', 'さんま', '半次', '上岡', '鉄平', 'ショージ', '山ちゃん', 'いかり', '昇太', '土田晃之']


# 入力 7
pp. 61-62

In [11]:

users = ['asahi', 'mainichi','Yomiuri_Online', 'nikkei',
 'nikkan_gendai', 'sakigake', 'toonippo']
for sn in users:
    feature_vector[sn] = tweetanalysis.get_feature_vector(
        tweetanalysis.get_tweets_score(sn, expectation, model)[0],
        model
    )

In [12]:
import tweetanalysis, numpy as np

users = ['asahi', 'mainichi','Yomiuri_Online', 'nikkei',
 'nikkan_gendai', 'sakigake', 'toonippo']
users2 = ['朝','毎', '読', '経', 'ゲ', '秋', '東奥']

# ベクトルのlistを2次元配列に格納
vectors = np.array([
    tweetanalysis.normalize(feature_vector[u]) for u in users
])

# コサイン類似度の行列を計算
news_similarity = pd.DataFrame(
    np.einsum('ik,jk->ij', vectors, vectors),
    columns = users2,
    index = users2
)
display(news_similarity)

,朝,毎,読,経,ゲ,秋,東奥
朝,1.000000,0.971345,0.941197,0.778815,0.881053,0.763461,0.758345
毎,0.971345,1.000000,0.944885,0.821602,0.884132,0.757335,0.735295
読,0.941197,0.944885,1.000000,0.775966,0.844696,0.786891,0.784721
経,0.778815,0.821602,0.775966,1.000000,0.669701,0.649386,0.587322
ゲ,0.881053,0.884132,0.844696,0.669701,1.000000,0.613522,0.618513
秋,0.763461,0.757335,0.786891,0.649386,0.613522,1.000000,0.976752
東奥,0.758345,0.735295,0.784721,0.587322,0.618513,0.976752,1.000000


# 入力 8
p. 63

In [13]:
# vectors.mean(0)が全新聞社の平均特徴量ベクトル
# 引いた結果をそれぞれ正規化
vectors2 = tweetanalysis.normalize(vectors - vectors.mean(0), 1)

# 原点を移動したベクトルにアクセスしやすくなるようにdictを準備
feature_vector2 = {k:v for k,v in zip(users, vectors2)}


# コサイン類似度の行列を計算
news_similarity2 = pd.DataFrame(
    np.einsum('ik,jk->ij', vectors2, vectors2),
    columns = users2,
    index = users2
)
display(news_similarity2)

,朝,毎,読,経,ゲ,秋,東奥
朝,1.000000,0.613484,0.257842,-0.183654,0.301299,-0.608063,-0.526931
毎,0.613484,1.000000,0.278400,0.078026,0.314666,-0.702628,-0.739747
読,0.257842,0.278400,1.000000,-0.182154,0.050742,-0.404511,-0.314495
経,-0.183654,0.078026,-0.182154,1.000000,-0.197683,-0.296688,-0.458325
ゲ,0.301299,0.314666,0.050742,-0.197683,1.000000,-0.621898,-0.522201
秋,-0.608063,-0.702628,-0.404511,-0.296688,-0.621898,1.000000,0.906468
東奥,-0.526931,-0.739747,-0.314495,-0.458325,-0.522201,0.906468,1.000000


# 入力 9


In [14]:
# ベクトルの次元
M = 50

# 引き去りたいベクトルの数 N<M
N = 5

# N個のM次元ベクトルをランダムで生成
W = np.random.normal(size=(M,N))

# 元のベクトルランダムで生成
V1 = np.random.normal(size=M)

# QR分解 デフォルトではQは N 列目までの部分のみ返される。
q,r = np.linalg.qr(W)

# Qのそれぞれの列ベクトル成分と内積をとって、
# それをそれぞれの列ベクトルに足した成分を引いている。
# V2がW[:,x]とそれぞれ独立(内積が0=>コサイン類似度が0)なベクトルとなっている。
V2 = V1 - np.einsum('i,ij,kj->k', V1, q, q)

import tweetanalysis

# コサイン類似度を計算してみる。
W = tweetanalysis.normalize( W)
V1 = tweetanalysis.normalize(V1)
V2 = tweetanalysis.normalize(V2)

print("V1とWの各ベクトルのコサイン類似度")
print(V1.dot(W))

print("V2とWの各ベクトルのコサイン類似度")
print(V2.dot(W))

V1とWの各ベクトルのコサイン類似度
[ 0.10449215  0.08208558 -0.10198595 -0.05087476  0.01623713]
V2とWの各ベクトルのコサイン類似度
[ -3.51281504e-17  -1.23065417e-17  -9.19403442e-17   7.89299182e-17
   5.20417043e-18]


# 入力 10

In [15]:
# 通常の mosts_imilar 関数による類似語
print("most_similar")
print(
    model.wv.most_similar([
        feature_vector2['asahi'] + feature_vector2['Yomiuri_Online']
    ])
)

# 直交成分からの類似語
print("orthogoal_words")
display(
    tweetanalysis.orthogonal_words(
        feature_vector2['Yomiuri_Online'] + feature_vector2['asahi'],
        model
     )
)

most_similar
[('流れ弾', 0.6762729287147522), ('自動車事故', 0.6698358654975891), ('結審', 0.6674159169197083), ('事故', 0.6670531034469604), ('凶行', 0.6590184569358826), ('事故死', 0.658760666847229), ('自供', 0.6572832465171814), ('銃撃', 0.6542375683784485), ('ギロチン', 0.6534814834594727), ('負傷', 0.6520446538925171)]
orthogoal_words


[('流れ弾', 0.6762729287147522),
 ('離婚', 0.5758596062660217),
 ('赤の広場', 0.5547539591789246),
 ('特別ルール', 0.5496065020561218),
 ('ハウツー', 0.5097163319587708),
 ('列車', 0.5745046138763428),
 ('ガルバルディ', 0.5364318490028381),
 ('取り下げ', 0.5334466099739075),
 ('欲鬼', 0.49935439229011536),
 ('13センチ', 0.5212986469268799)]